<a href="https://colab.research.google.com/github/UNT-CSCE-Project/sentiment-classification-of-amazon-reviews/blob/dev_avijeet/Copy_of_sentiment_classification_amazon_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00


In [46]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.6 MB/s eta 0:00:00


In [1]:

### Connect with google drive

### import csv from google dir
def connect_and_fetch_file():
  from google.colab import drive
  drive.mount('/content/drive')

  file_id = "1ZBezHHtxu8L6pZVeOtaC0-_n3MjOYnGR"

  file_dir = f'https://drive.google.com/uc?id={file_id}'
  return file_dir
file_dir = connect_and_fetch_file()



Mounted at /content/drive


In [103]:
import pandas as pd

df = pd.read_csv(file_dir)


<ipython-input-103-d4830fe90155>:3: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_dir)


In [104]:
df_updated = df.copy()  # Create a copy to keep the original DataFrame
df_updated.columns = [col[len('reviews.'):] if col.startswith('reviews.') else col for col in df.columns]


In [117]:
df_updated=df_updated[[ 'doRecommend',  'numHelpful', 'rating',  'text', 'title']]

In [118]:
from textblob import TextBlob

def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

def sentiment_scores(text):
    sentiment = analyze_sentiment(text)
    if sentiment > 0.5:
        return 5
    elif 0.2 < sentiment <= 0.5:
        return 4
    elif -0.2 <= sentiment <= 0.2:
        return 3
    elif -0.5 <= sentiment < -0.2:
        return 2
    else:
        return 1



In [119]:
sentiments = df_updated[df_updated['doRecommend'].isna() & df_updated['rating'].isna()]['text'].apply(sentiment_scores)

In [120]:
df_updated['rating'].fillna(df_updated[df_updated['doRecommend'].isna() & df_updated['rating'].isna()]['text'].apply(sentiment_scores), inplace=True)

In [121]:
df_updated['text' ].fillna('Five Stars', inplace=True)

In [122]:
df_updated['doRecommend'].fillna(False, inplace=True)
df_updated['numHelpful'].fillna(0, inplace=True)

In [123]:
df_updated[df_updated['title'].isna()].iloc[:]

,doRecommend,numHelpful,rating,text,title
3083,False,0,5,A must have item.,NaN
21953,True,1,5,I like it. I use mostly for music.,NaN
25515,True,0,5,I was totally stoked about this product becaus...,NaN
28578,True,0,5,I'm in LOVE with ALEXA!!,NaN
28680,True,1,5,"Love it ! I started with a kindle,then a fire ...",NaN


In [124]:
df_updated['numHelpful'] = df_updated['numHelpful'].astype('int')
df_updated['rating'] = df_updated['rating'].astype('int')

,doRecommend,numHelpful,rating,text,title
3083,False,0,5,A must have item.,NaN
21953,True,1,5,I like it. I use mostly for music.,NaN
25515,True,0,5,I was totally stoked about this product becaus...,NaN
28578,True,0,5,I'm in LOVE with ALEXA!!,NaN
28680,True,1,5,"Love it ! I started with a kindle,then a fire ...",NaN


In [129]:
df_updated = df_updated.drop('title', axis=1)

In [130]:
df_updated

,doRecommend,numHelpful,rating,text
0,True,0,5,This product so far has not disappointed. My c...
1,True,0,5,great for beginner or experienced person. Boug...
2,True,0,5,Inexpensive tablet for him to use and learn on...
3,True,0,4,I've had my Fire HD 8 two weeks now and I love...
4,True,0,5,I bought this for my grand daughter when she c...
...,...,...,...,...
34655,False,0,3,This is not appreciably faster than any other ...
34656,False,0,1,Amazon should include this charger with the Ki...
34657,False,0,1,Love my Kindle Fire but I am really disappoint...
34658,False,0,1,I was surprised to find it did not come with a...
